<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_23_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [ ]:
DB_USER = "prog_academy_da_yuq6_user"
DB_PASS = "nKLaf3orGpgat1GEpFbkuNVtxcNAGJp1"
DB_HOST = "dpg-d519e97pm1nc73briqk0-a.oregon-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "prog_academy_da_yuq6"

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

У нас есть пользователи, которые:

* устанавливают приложение (`app_sessions`),
* смотрят товары (`product_views_log`),
* авторизуются (`devices_users_map`),
* совершают покупки (`orders_log`).

# Работа с датами в SQL
**Основные функции для работы с датой для анализа данных**

In [ ]:
query = "SELECT * FROM app_sessions LIMIT 5;"
df = pd.read_sql(text(query), engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00001,DVC0001,2024-01-15,Android,Referral,13.91
2,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
3,S00003,DVC0003,2024-02-23,iOS,Referral,14.98
4,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64


## `DATE_TRUNC`  
**обрезает дату или время до заданной единицы измерения**, обнуляя менее значимые компоненты.

### Общий синтаксис:

```sql
DATE_TRUNC('единица_времени', timestamp)
```

---

### Для чего применяем:

1. **Группировка по времени**:
   Чтобы сгруппировать данные по неделям, месяцам, дням и т.п.
2. **Округление до начала часа, дня и т.п.**
3. **Сравнение дат на уровне нужной точности** (например, все события за конкретный месяц).

---

### Поддерживаемые единицы:

* `millennium`, `century`, `decade`, `year`, `quarter`, `month`
* `week`, `day`, `hour`, `minute`, `second`

In [ ]:
query = """
SELECT DATE_TRUNC('day', first_seen) FROM app_sessions
LIMIT 5;
"""
df = pd.read_sql(text(query), engine)
df

,date_trunc
0,2024-04-25 00:00:00+00:00
1,2024-01-15 00:00:00+00:00
2,2024-01-18 00:00:00+00:00
3,2024-02-23 00:00:00+00:00
4,2024-04-24 00:00:00+00:00


In [ ]:
query = """
SELECT DATE_TRUNC('month', first_seen) FROM app_sessions
LIMIT 5;
"""
df = pd.read_sql(text(query), engine)
df

,date_trunc
0,2024-04-01 00:00:00+00:00
1,2024-01-01 00:00:00+00:00
2,2024-01-01 00:00:00+00:00
3,2024-02-01 00:00:00+00:00
4,2024-04-01 00:00:00+00:00


In [ ]:
query = """
SELECT DATE_TRUNC('year', first_seen) FROM app_sessions
LIMIT 5;
"""
df = pd.read_sql(text(query), engine)
df

,date_trunc
0,2024-01-01 00:00:00+00:00
1,2024-01-01 00:00:00+00:00
2,2024-01-01 00:00:00+00:00
3,2024-01-01 00:00:00+00:00
4,2024-01-01 00:00:00+00:00


## `EXTRACT`
**выделение отдельной компоненты даты или времени**, например: год, месяц, день, час и т.д.

### Синтаксис:

```sql
EXTRACT(единица_времени FROM дата_или_время)
```

---

### Используемые единицы:

* `YEAR`
* `MONTH`
* `DAY`
* `HOUR`
* `MINUTE`
* `SECOND`
* `DOW` (day of week: 0 = Sunday, 6 = Saturday)
* `DOY` (day of year)
* `WEEK`
* `QUARTER`

---

### Для чего применяем:

1. **Фильтрация по части даты**:
2. **Создание временных группировок**:

---

### Разница с `DATE_TRUNC`:

|                | `EXTRACT`                     | `DATE_TRUNC`                                 |
| -------------- | ----------------------------- | -------------------------------------------- |
| Что делает     | Возвращает одно число         | Обнуляет менее значимые части даты           |
| Тип результата | `numeric`                     | `timestamp`                                  |
| Пример         | `EXTRACT(MONTH FROM d)` → `7` | `DATE_TRUNC('month', d)` → `'2025-07-01...'` |

In [ ]:
query = """
SELECT EXTRACT(YEAR FROM first_seen) FROM app_sessions
LIMIT 5;
"""

df = pd.read_sql(text(query), engine)
df

,extract
0,2024.0
1,2024.0
2,2024.0
3,2024.0
4,2024.0


In [ ]:
query = """
SELECT EXTRACT(MONTH FROM first_seen) FROM app_sessions
LIMIT 5;
"""

df = pd.read_sql(text(query), engine)
df

,extract
0,4.0
1,1.0
2,1.0
3,2.0
4,4.0


In [ ]:
query = """
SELECT EXTRACT(DOW FROM first_seen) FROM app_sessions
LIMIT 5;
"""

df = pd.read_sql(text(query), engine)
df

,extract
0,4.0
1,1.0
2,4.0
3,5.0
4,3.0


## `CURRENT_DATE`, `CURRENT_TIMESTAMP` и `NOW()`
**текущее время и/или дата**

### Особенности

| Функция             | Что возвращает                 | Тип данных  | Пример результата            |
| ------------------- | ------------------------------ | ----------- | ---------------------------- |
| `CURRENT_DATE`      | Только текущую дату            | `DATE`      | `2025-07-14`                 |
| `CURRENT_TIMESTAMP` | Текущие дата **и** время       | `TIMESTAMP` | `2025-07-14 11:25:32.123456` |
| `NOW()`             | То же, что `CURRENT_TIMESTAMP` | `TIMESTAMP` | `2025-07-14 11:25:32.123456` |

In [ ]:
query = """
SELECT CURRENT_DATE;
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)
    print(type(result))
    print(result[0])

(datetime.date(2025, 7, 14),)
<class 'sqlalchemy.engine.row.Row'>
2025-07-14


In [ ]:
query = """
SELECT CURRENT_TIMESTAMP;
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)
    print(type(result))
    print(result[0])

(datetime.datetime(2025, 7, 14, 16, 43, 6, 250581, tzinfo=datetime.timezone.utc),)
<class 'sqlalchemy.engine.row.Row'>
2025-07-14 16:43:06.250581+00:00


In [ ]:
query = """
SELECT NOW();
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)
    print(type(result))
    print(result[0])

(datetime.datetime(2025, 7, 14, 16, 43, 52, 424084, tzinfo=datetime.timezone.utc),)
<class 'sqlalchemy.engine.row.Row'>
2025-07-14 16:43:52.424084+00:00


## `INTERVAL`  
 **Прибавление/вычитание временного промежутка**: дни, часы, месяцы, секунды и т.д.

### 📌 Синтаксис:

```sql
SELECT CURRENT_DATE + INTERVAL '7 days';
```
---

## Единицы измерения:

* `second`, `minute`, `hour`
* `day`, `week`, `month`, `year`
* И даже комбинации:

```sql
INTERVAL '2 days 3 hours 15 minutes'
```
---

## Особенности:

* `INTERVAL` сам по себе не дата — это просто **временной отрезок**.
* Может использоваться и с `DATE`, и с `TIMESTAMP`.
* В некоторых СУБД, например MySQL, `INTERVAL` пишется иначе:

```sql
-- MySQL стиль
SELECT NOW() + INTERVAL 1 DAY;
```
---

### В Pandas аналог:

```python
import pandas as pd

pd.Timestamp.now() + pd.Timedelta(days=3, hours=2)
```

In [ ]:
query = """
SELECT CURRENT_DATE + INTERVAL '7 day';
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)

(datetime.datetime(2025, 7, 21, 0, 0),)


In [ ]:
query = """
SELECT CURRENT_DATE + INTERVAL '7 month';
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)

(datetime.datetime(2026, 2, 14, 0, 0),)


In [ ]:
query = """
SELECT CURRENT_DATE + INTERVAL '2 years 1 month';
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    result = result.fetchone()
    print(result)

(datetime.datetime(2027, 8, 14, 0, 0),)


In [ ]:
query = """
SELECT session_id, first_seen, first_seen - INTERVAL '1 day' AS minus_one_day FROM app_sessions;
"""

df = pd.read_sql(text(query), engine)
df

,session_id,first_seen,minus_one_day
0,S00000,2024-04-25,2024-04-24
1,S00001,2024-01-15,2024-01-14
2,S00002,2024-01-18,2024-01-17
3,S00003,2024-02-23,2024-02-22
4,S00004,2024-04-24,2024-04-23
...,...,...,...
495,S00495,2024-06-20,2024-06-19
496,S00496,2024-02-08,2024-02-07
497,S00497,2024-01-10,2024-01-09
498,S00498,2024-02-11,2024-02-10


## Когда применяем работу с датами и временем

Анализ по дате и времени нужен в ситуациях, когда мы хотим:

| Цель анализа                               | Когда применять                                               | Что нужно                         |
| ------------------------------------------ | ------------------------------------------------------------- | --------------------------------- |
| **1. Графики по дням / неделям / месяцам** | Отслеживать динамику показателей                              | Группировка по дате               |
| **2. Сезонность**                          | Найти циклы в поведении (например, пятничные пики покупок)    | Группировка по дню недели, месяцу |
| **3. Повторяемость покупок**               | Сколько дней проходит между покупками                         | Расчёт интервалов между датами    |
| **4. RFM-анализ**                          | Сегментировать пользователей по их активности и ценности      | DATEDIFF, COUNT, SUM              |

### 1. Графики **по дням, неделям, месяцам**

**Зачем**: смотреть динамику продаж, посещений, заказов во времени.

**Как**: используем `DATE_TRUNC`, `GROUP BY`, агрегаты.

In [ ]:
# Сколько установок в день
query = """
SELECT first_seen, COUNT(*) AS installs
FROM app_sessions
GROUP BY first_seen
ORDER BY first_seen;
"""

df = pd.read_sql(text(query), engine)
df

,first_seen,installs
0,2024-01-01,3
1,2024-01-02,1
2,2024-01-03,3
3,2024-01-04,4
4,2024-01-05,3
...,...,...
169,2024-06-24,1
170,2024-06-25,2
171,2024-06-26,3
172,2024-06-27,1


In [ ]:
fig = px.line(df, x="first_seen", y="installs")
fig.show()

In [ ]:
# Сколько заказов по неделям
query = """
SELECT DATE_TRUNC('week', order_time) AS week, COUNT(*) AS orders
FROM orders_log
GROUP BY week
ORDER BY week;
"""

df = pd.read_sql(text(query), engine)
df

,week,orders
0,2024-01-01 00:00:00+00:00,110
1,2024-01-08 00:00:00+00:00,99
2,2024-01-15 00:00:00+00:00,105
3,2024-01-22 00:00:00+00:00,95
4,2024-01-29 00:00:00+00:00,106
5,2024-02-05 00:00:00+00:00,111
6,2024-02-12 00:00:00+00:00,104
7,2024-02-19 00:00:00+00:00,92
8,2024-02-26 00:00:00+00:00,100
9,2024-03-04 00:00:00+00:00,98


In [ ]:
fig = px.line(df, x="week", y="orders")
fig.show()

In [ ]:
# Сколько просмотров в месяц
query = """
SELECT DATE_TRUNC('month', view_date) AS month, COUNT(*) AS views
FROM product_views_log
GROUP BY month
ORDER BY month;
"""

df = pd.read_sql(text(query), engine)
df

,month,views
0,2024-01-01 00:00:00+00:00,527
1,2024-02-01 00:00:00+00:00,536
2,2024-03-01 00:00:00+00:00,539
3,2024-04-01 00:00:00+00:00,534
4,2024-05-01 00:00:00+00:00,590
5,2024-06-01 00:00:00+00:00,507


In [ ]:
fig = px.line(df, x="month", y="views")
fig.show()

### 2. **Cезонность и циклы** (ежемесячная/еженедельная активность)

**Зачем**: находить пики и спады по времени года, неделе, дню недели.

**Как**: `EXTRACT(MONTH FROM date)`, `EXTRACT(DOW FROM date)`.

In [ ]:
# Сколько заказов по дням недели
query = """
SELECT EXTRACT(DOW FROM order_time) AS day_of_week, COUNT(*) AS orders
FROM orders_log
GROUP BY day_of_week
ORDER BY day_of_week;
"""

df = pd.read_sql(text(query), engine)
df

,day_of_week,orders
0,0.0,332
1,1.0,393
2,2.0,347
3,3.0,381
4,4.0,363
5,5.0,365
6,6.0,378


### 3. Повторяемость покупок

**Зачем**: определить, как часто пользователи возвращаются и сколько времени проходит между покупками.

**Как**: сравниваем `DATEDIFF` между заказами.

In [ ]:
# Сколько дней между первой и последней покупкой
query = """
SELECT user_uuid, MAX(order_time) - MIN(order_time) AS days_between_orders, COUNT(*) AS orders
FROM orders_log
GROUP BY user_uuid
HAVING COUNT(*) > 1
ORDER BY days_between_orders DESC;
"""

df = pd.read_sql(text(query), engine)
df

,user_uuid,days_between_orders,orders
0,USR0268,179,15
1,USR0125,177,13
2,USR0269,177,14
3,USR0272,176,15
4,USR0042,176,10
...,...,...,...
295,USR0191,19,2
296,USR0096,15,2
297,USR0130,11,3
298,USR0132,1,2


In [ ]:
# Связь между количеством заказов и "жизненным циклом" пользователя
fig = px.scatter(df, x="orders", y="days_between_orders")
fig.show()

In [ ]:
# сколько пользователей возвращаются через X дней?
fig = px.histogram(df, x="days_between_orders")
fig.show()

### 4. Анализ RFM (Recency, Frequency, Monetary)

**Зачем**: **метод сегментации клиентов** по их поведению в отношении заказов, который помогает понять ценность клиента и его лояльность.


#### Что означает каждая буква?

| Параметр      | Что измеряет                    | Пример расчета                                            |
| ------------- | ------------------------------- | --------------------------------------------------------- |
| **Recency**   | Как давно был последний заказ   | Разница между сегодняшней датой и датой последнего заказа |
| **Frequency** | Как часто клиент делает покупки | Количество заказов за период                              |
| **Monetary**  | Сколько денег клиент принес     | Сумма всех заказов                                        |

---

#### Для чего используется RFM?

* **Сегментация**: найти VIP-клиентов, "спящих" клиентов, новых и т.д.
* **Персонализация**: настроить email-маркетинг или push-уведомления
* **Предсказание оттока**: кто может уйти

**Задача — оценить активность клиентов**:
* Как давно был последний заказ (**Recency**),
* Сколько всего заказов сделал клиент (**Frequency**),
* На какую сумму клиент потратился (**Monetary**).


In [ ]:
# Запрос для расчета RFM-показателей
query = """
SELECT
  user_uuid,
  CURRENT_DATE - MAX(order_time) AS recency,
  COUNT(*) AS frequency,
  SUM(total_uah) AS monetary
FROM orders_log
WHERE total_uah IS NOT NULL
GROUP BY user_uuid;
"""

rfm = pd.read_sql(text(query), engine)
rfm

,user_uuid,recency,frequency,monetary
0,USR0107,651,5,4947.18
1,USR0066,558,13,19064.47
2,USR0106,546,14,19179.84
3,USR0224,580,9,18292.62
4,USR0138,556,14,18920.32
...,...,...,...,...
295,USR0231,552,13,23953.07
296,USR0008,545,15,23048.32
297,USR0104,552,14,24700.33
298,USR0023,561,13,17250.25


In [ ]:
# Recency: чем меньше дней — тем лучше (инверсия)
rfm["R"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])

# Frequency и Monetary: чем больше — тем лучше
rfm["F"] = pd.qcut(rfm["frequency"], 5, labels=[1,2,3,4,5])
rfm["M"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

# приводим к int
rfm[["R","F","M"]] = rfm[["R","F","M"]].astype(int)


In [ ]:
rfm

,user_uuid,recency,frequency,monetary,R,F,M
0,USR0107,651,5,4947.18,1,2,1
1,USR0066,558,13,19064.47,3,4,4
2,USR0106,546,14,19179.84,5,5,4
3,USR0224,580,9,18292.62,2,3,4
4,USR0138,556,14,18920.32,3,5,4
...,...,...,...,...,...,...,...
295,USR0231,552,13,23953.07,4,4,5
296,USR0008,545,15,23048.32,5,5,5
297,USR0104,552,14,24700.33,4,5,5
298,USR0023,561,13,17250.25,3,4,4


**Recency (давность последней покупки)**

```python
rfm["R"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1]).astype(int)
```

* `pd.qcut()` делит `recency` на **5 квантилей** (примерно равные по размеру группы).
* `labels=[5, 4, 3, 2, 1]` — меньшее значение `recency` (то есть недавняя покупка) получает **лучший балл 5**.
* То есть: покупал вчера → `recency = 1` → `R = 5`

*Почему обратный порядок (5, 4, ..., 1)?*

Потому что чем **меньше дней прошло с последней покупки**, тем **ценнее клиент**.

---

**Frequency (частота покупок)**

```python
rfm["F"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]).astype(int)
```

* `rank(method="first")` присваивает уникальные ранги (если значения одинаковые).
* Затем `pd.qcut(..., 5)` разбивает их на 5 равных групп.
* `labels=[1, 2, 3, 4, 5]` — чаще покупает → выше `frequency` → выше балл `F`

---

**Monetary (сколько потратил)**

```python
rfm["M"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5]).astype(int)
```

* Всё просто: чем больше тратит, тем выше балл.
* `qcut` делит пользователей по объёму трат на 5 групп от 1 (мало) до 5 (много).

---

**Сборка итогового RFM-скора**

```python
rfm["RFM_score"] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)
```

* Склеивает значения `R`, `F` и `M` в строку, например:
  `R=5`, `F=3`, `M=4` → `RFM_score = "534"`

---

**В результате:**  

| user\_id | R | F | M | RFM\_score |
| -------- | - | - | - | ---------- |
| 101      | 5 | 3 | 4 | "534"      |
| 102      | 2 | 5 | 5 | "255"      |

---

In [ ]:
def segment(row):

    # 1. Лучшие клиенты
    if row["R"] >= 4 and row["F"] >= 4 and row["M"] >= 4:
        return "VIP"

    # 2. Лояльные
    if row["R"] >= 3 and row["F"] >= 4:
        return "Loyal"

    # 3. Новые
    if row["F"] == 1 and row["R"] >= 4:
        return "New"

    # 4. Потенциально лояльные
    if row["R"] >= 4 and row["F"] in [2, 3]:
        return "Potential Loyalist"

    # 5. Под риском
    if row["R"] <= 2 and row["F"] <= 3 and row["M"] >= 3:
        return "At Risk"

    # 6. Потерянные VIP
    if row["R"] <= 2 and row["F"] >= 4 and row["M"] >= 4:
        return "Lost VIP"

    # 7. Спящие
    if row["R"] == 3 and row["F"] <= 2:
        return "Sleeping"

    # 8. Остальные
    return "Regular"

rfm["segment"] = rfm.apply(segment, axis=1)
rfm


,user_uuid,recency,frequency,monetary,R,F,M,segment
0,USR0107,651,5,4947.18,1,2,1,Regular
1,USR0066,558,13,19064.47,3,4,4,Loyal
2,USR0106,546,14,19179.84,5,5,4,VIP
3,USR0224,580,9,18292.62,2,3,4,At Risk
4,USR0138,556,14,18920.32,3,5,4,Loyal
...,...,...,...,...,...,...,...,...
295,USR0231,552,13,23953.07,4,4,5,VIP
296,USR0008,545,15,23048.32,5,5,5,VIP
297,USR0104,552,14,24700.33,4,5,5,VIP
298,USR0023,561,13,17250.25,3,4,4,Loyal


In [ ]:
color_map = {
    "VIP": "#D62728",               # красный
    "Loyal": "#FF7F0E",             # оранжевый
    "Potential Loyalist": "#1F77B4",# синий
    "New": "#9467BD",               # фиолетовый
    "Regular": "#7F7F7F",           # серый
    "At Risk": "#2CA02C",           # зелёный
    "Lost VIP": "#8C564B",          # коричневый
    "Sleeping": "#AEC7E8"            # светло-синий
}

fig = px.scatter(
    rfm,
    x="frequency",
    y="monetary",
    color="segment",
    color_discrete_map=color_map,
    title="RFM: Frequency vs Monetary",
    hover_data=["recency", "R", "F", "M"]
)

fig.show()


In [ ]:
seg_dist = rfm["segment"].value_counts().reset_index()
seg_dist.columns = ["segment", "count"]

fig = px.bar(
    seg_dist,
    x="segment",
    y="count",
    title="Customer Distribution by RFM Segment",
    text="count"
)

fig.show()
